# Running the model

Setup a `feisty` integration.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import feisty
import matplotlib.pyplot as plt
import numpy as np
import yaml

## Configure testcase to run

TODO: this is using the first column (which I believe is shallowest); want to add deepest column as well.
That will require refactoring the yaml file and also changing test_ds.

In [3]:
testcase = feisty.driver.simulate_testcase("tanh_shelf", "cyclic")
# print(testcase.obj.biomass.isel(X=0).sel(group='benthic_prey').data)
with open("matlab_first_timestep.yaml") as f:
    baselines = yaml.safe_load(f)

In [4]:
def compare(baselines_dict, da, full_table=True, markdown_formatting=False):
    if markdown_formatting:
        print(f"| {baselines_dict['dimname']} | Matlab Value | Python Value | Rel Err |")
        print("| --- | --- | --- | --- |")
        seps = ['| ', ' | ', ' | ', ' |']
    else:
        seps = ['', ': ', ', ', '']
    for var in da[baselines_dict['dimname']].data:
        sel_dict = {baselines_dict['dimname']: var}
        matlab_val = baselines_dict['values'][var]
        py_val = da.sel(sel_dict).data
        if matlab_val != 0:
            rel_err = (py_val - matlab_val) / matlab_val
            if rel_err < 0:
                rel_err = -rel_err
        else:
            if py_val == 0:
                rel_err = 0
            elif py_val > 0:
                rel_err = np.inf
            elif py_val < 0:
                rel_err = -np.inf
        if full_table or rel_err > 1e-2:
            print(f"{seps[0]}{var}{seps[1]}{matlab_val:10.4e}{seps[2]}{py_val:10.4e}{seps[2]}{rel_err:10.4e}{seps[-1]}")

## Run the model


In [5]:
%%time

nsteps = 1
testcase.run(nsteps)
# print(testcase.ds['feeding_link'].data)
testcase.ds.info


CPU times: user 269 ms, sys: 5.26 ms, total: 275 ms
Wall time: 636 ms


<bound method Dataset.info of <xarray.Dataset>
Dimensions:                    (X: 22, group: 9, time: 1, fish: 8, feeding_link: 19)
Coordinates:
  * X                          (X) float64 -0.5 -0.2381 0.02381 ... 4.738 5.0
  * group                      (group) <U12 'Sf' 'Sp' ... 'Ld' 'benthic_prey'
  * fish                       (fish) <U2 'Sf' 'Sp' 'Sd' 'Mf' ... 'Md' 'Lp' 'Ld'
  * feeding_link               (feeding_link) <U15 'Sf_Zoo' ... 'Ld_benthic_p...
    predator                   (feeding_link) <U2 'Sf' 'Sp' 'Sd' ... 'Ld' 'Ld'
    prey                       (feeding_link) <U12 'Zoo' ... 'benthic_prey'
Dimensions without coordinates: time
Data variables: (12/16)
    biomass                    (time, group, X) float64 9.998e-06 ... 1e-05
    T_habitat                  (time, fish, X) float64 18.2 18.2 ... 18.2 18.2
    ingestion_rate             (time, fish, X) float64 0.2152 ... 6.439e-07
    predation_flux             (time, fish, X) float64 5.726e-12 ... 0.0
    predation_rat

In [6]:
if (nsteps > 1):
    test_ds = testcase.ds.isel(X=0,time=0)
    for group in test_ds.group.data:
        testcase.ds.biomass.sel(group=group).isel(X=0).plot()
    plt.ylim([1e-6, 1])
    plt.yscale("log")
    plt.legend(test_ds.group.data)
else:
    test_ds = testcase.ds.isel(X=0, time=0)

In [7]:
compare(baselines["encounter"], test_ds["encounter_rate_link"], markdown_formatting=True)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Sp_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Sd_Zoo | 2.2885e+00 | 2.2936e+00 | 2.2411e-03 |
| Mf_Zoo | 2.9714e-01 | 2.9781e-01 | 2.2411e-03 |
| Mf_Sf | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mf_Sp | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mf_Sd | 1.9837e-06 | 0.0000e+00 | 1.0000e+00 |
| Mp_Zoo | 2.9714e-01 | 2.9781e-01 | 2.2411e-03 |
| Mp_Sf | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mp_Sp | 1.9837e-06 | 1.9836e-06 | 3.9203e-05 |
| Mp_Sd | 1.9837e-06 | 0.0000e+00 | 1.0000e+00 |
| Md_benthic_prey | 1.7232e-04 | 1.7232e-04 | 2.6443e-05 |
| Lp_Mf | 5.7237e-07 | 2.8617e-07 | 5.0002e-01 |
| Lp_Mp | 5.7237e-07 | 5.7235e-07 | 3.9203e-05 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 2.3891e-07 | 8.9590e-08 | 6.2500e-01 |
| Ld_Mp | 2.3891e-07 | 1.7918e-07 | 2.5000e-01 |
| Ld_Md | 2.3891e-07 | 2.3891e-07 | 2.0168e-06 |
| Ld_benthic_prey | 

In [8]:
compare(baselines["consumption"], test_ds["consumption_rate_link"], markdown_formatting=True)

| feeding_link | Matlab Value | Python Value | Rel Err |
| --- | --- | --- | --- |
| Sf_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sp_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Sd_Zoo | 2.1520e-01 | 2.1523e-01 | 1.7479e-04 |
| Mf_Zoo | 4.2967e-02 | 4.2980e-02 | 2.9564e-04 |
| Mf_Sf | 2.8684e-07 | 2.8628e-07 | 1.9803e-03 |
| Mf_Sp | 2.8684e-07 | 2.8628e-07 | 1.9803e-03 |
| Mf_Sd | 2.8684e-07 | 0.0000e+00 | 1.0000e+00 |
| Mp_Zoo | 4.2967e-02 | 4.2980e-02 | 2.9564e-04 |
| Mp_Sf | 2.8684e-07 | 2.8628e-07 | 1.9803e-03 |
| Mp_Sp | 2.8684e-07 | 2.8628e-07 | 1.9803e-03 |
| Mp_Sd | 2.8684e-07 | 0.0000e+00 | 1.0000e+00 |
| Md_benthic_prey | 1.7091e-04 | 1.7090e-04 | 2.6241e-05 |
| Lp_Mf | 5.7231e-07 | 2.8615e-07 | 5.0001e-01 |
| Lp_Mp | 5.7231e-07 | 5.7230e-07 | 1.2265e-05 |
| Lp_Md | 0.0000e+00 | 0.0000e+00 | 0.0000e+00 |
| Ld_Mf | 2.3620e-07 | 8.8578e-08 | 6.2498e-01 |
| Ld_Mp | 2.3620e-07 | 1.7716e-07 | 2.4997e-01 |
| Ld_Md | 2.3620e-07 | 2.3621e-07 | 4.4872e-05 |
| Ld_benthic_prey | 

In [ ]:
test_ds.sel(feeding_link='Sd_Zoo')["encounter_rate_link"].data

In [ ]:
test_ds.sel(feeding_link='Mf_Sd')["encounter_rate_link"].data

In [ ]:
test_ds["encounter_rate_total"]

In [ ]:
test_ds["predation_flux"].sel(fish="Sd")

In [ ]:
test_ds.sel(fish='Sd')['consumption_rate_max_pred']

In [ ]:
test_ds.sel(feeding_link='Lp_Md')["encounter_rate_link"]

In [ ]:
testcase.ds.sel(feeding_link='Sf_Zoo')["consumption_rate_link"]